In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [155]:
os.chdir('C:/Storage/StudyJourney/Nam3/Cs114/Project2')

In [156]:
# Đọc dữ liệu
annonimized = pd.read_csv("Dataset/annonimized.csv")
th = pd.read_csv("Dataset/public_it001/th-public.csv")

In [157]:
# Merge dữ liệu: giữ tất cả các username từ annonimized
data = annonimized.merge(th, on="username", how="left")

# Phân tách dữ liệu
train_test_data = data[data['TH'].notna()]  # Sinh viên có điểm, dùng để train
predict_data = data[data['TH'].isna()]  # Sinh viên không có điểm, dùng để dự đoán

# Xử lý dữ liệu
# Xóa cột không cần thiết
train_test_data.drop(['assignment_id', 'problem_id'], axis=1, inplace=True)

# In số lượng
print(f"Số lượng sinh viên có điểm (train): {train_test_data.shape[0]}")
print(f"Số lượng sinh viên cần dự đoán: {predict_data.shape[0]}")

Số lượng sinh viên có điểm (train): 156973
Số lượng sinh viên cần dự đoán: 138225


C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\3577564136.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data.drop(['assignment_id', 'problem_id'], axis=1, inplace=True)


In [158]:
train_test_data

,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement,TH
458,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,10/11/2025 16:15,10/11/2025 16:16,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd...",4.5
459,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,10/11/2025 16:16,10/11/2025 16:16,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd...",4.5
460,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,10/11/2025 16:23,10/11/2025 16:43,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd...",4.5
461,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,10/11/2025 16:25,10/11/2025 16:25,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd...",4.5
462,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,10/11/2025 16:26,10/11/2025 16:26,"{""times"":[],""mems"":[],""verdicts"":{""\n\nforbidd...",4.5
...,...,...,...,...,...,...,...,...,...,...
295193,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,1/15/2025 16:03,1/15/2025 16:03,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",9
295194,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,1/15/2025 16:04,1/15/2025 16:05,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",9
295195,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,1/15/2025 16:04,1/15/2025 16:05,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",9
295196,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,1/15/2025 16:05,1/15/2025 16:05,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",9


In [159]:
print(train_test_data['status'].unique())

['Compilation Error' 'SCORE' 'Syntax Error' 'pending']


In [160]:
print(train_test_data.dtypes)

username       object
is_final        int64
status         object
pre_score       int64
coefficient     int64
language_id    object
created_at     object
updated_at     object
judgement      object
TH             object
dtype: object


In [161]:
# Nhóm theo 'username' và tính toán các đặc trưng tổng hợp
# Mã hóa dữ liệu
label_encoder = LabelEncoder()

# train_test_data['status_encoded'] = label_encoder.fit_transform(train_test_data['status'])

status_mapping = {
    "Compilation Error": 0,
    "Syntax Error": 0,
    "pending": 1,
    "SCORE": 2
}

# Áp dụng ánh xạ
train_test_data['status_encoded'] = train_test_data['status'].map(status_mapping)

train_test_data['submission_count'] = train_test_data.groupby('username')['is_final'].transform('count')
train_test_data['avg_pre_score'] = train_test_data.groupby('username')['pre_score'].transform('mean')

train_test_data['avg_score_per_submission'] = train_test_data['avg_pre_score'] / (train_test_data['submission_count'] + 1e-5)  # Tránh chia cho 0

train_test_data['TH'] = pd.to_numeric(train_test_data['TH'], errors='coerce')

# Nhóm theo 'username' và tính toán các đặc trưng tổng hợp
agg_data = train_test_data.groupby('username').agg({
    'avg_pre_score': 'mean',  # Trung bình điểm trước-
    'submission_count': 'mean',  # Trung bình thời gian nộp
    'avg_score_per_submission': 'mean', # Trung bình điểm trên tổng số lần nộp
    'status_encoded': 'mean',  # Trung bình trạng thái (hoặc có thể tính số lần xuất hiện nếu cần)
    'is_final': lambda x: (x == 1).sum()  # Đếm số lần giá trị bằng 1 trong cột is_final
}).reset_index()

# Cập nhật bộ dữ liệu với các đặc trưng tổng hợp
agg_data['TH'] = train_test_data.groupby('username')['TH'].mean().values

# Tiếp theo, bạn có thể thực hiện các bước xử lý dữ liệu, chuẩn hóa và chia tách bộ dữ liệu như sau
features = ['status_encoded', 'is_final', 'submission_count', 'avg_pre_score', 'avg_score_per_submission']
# features = ['username','status_encoded', 'is_final', 'submission_count', 'avg_pre_score', 'avg_score_per_submission']

# Xử lý chuẩn hóa dữ liệu
X = agg_data[features]
y = agg_data['TH']

scaler = StandardScaler()
X = scaler.fit_transform(X)

# fill missing value
y.fillna(0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1728323246.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data['status_encoded'] = train_test_data['status'].map(status_mapping)
C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1728323246.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data['submission_count'] = train_test_data.groupby('username')['is_final'].transform('count')
C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1728323246.py:18: SettingWithCopyWarning: 
A

In [162]:
X

array([[ 0.0764792 , -0.61049816, -0.43788962,  0.25104624, -0.12513821],
       [ 0.05650578,  0.34596611,  0.36727578,  0.24048343, -0.14716827],
       [ 0.75864828,  0.01718152, -0.09281874,  0.56906712, -0.13436748],
       ...,
       [-0.3478593 , -0.4012716 , -0.38037781, -0.60010771, -0.13874957],
       [-0.40978569,  1.45187794,  2.50959085, -0.57890544, -0.16548135],
       [-1.1757432 , -1.26806736, -1.10646446, -0.46659724, -0.06432691]])

In [163]:
X_train

array([[ 0.11124694,  1.06331432,  1.79069318, -0.48046318, -0.1627717 ],
       [-1.34421631,  0.79430875,  1.00709471, -1.25153026, -0.16309666],
       [ 0.43069745, -0.16215553, -0.45226757,  0.45776031, -0.12158455],
       ...,
       [-3.61860329, -0.0425975 , -0.63199199, -3.10395587, -0.16968106],
       [ 1.12020213,  0.25629759,  0.43197657,  0.45069672, -0.14656289],
       [-0.68539348,  0.73452973,  0.59732303, -0.41572811, -0.15484088]])

In [164]:
y_train

570     8.0
235     8.5
76     10.0
275     8.0
72      9.5
       ... 
71      1.0
106     1.0
270     0.5
435     9.5
102     8.5
Name: TH, Length: 679, dtype: float64

In [165]:
X_test

array([[-0.74544635,  0.55519268,  1.10055141, -0.78800668, -0.16098738],
       [-0.12157722,  1.15298285,  0.30257499,  0.51879789, -0.14390579],
       [ 0.28112992,  0.40574513, -0.09281874,  0.70836411, -0.13293762],
       [ 0.4939646 ,  0.07696054,  1.6612916 , -0.48822341, -0.16226904],
       [-0.62517129, -1.0887303 , -0.85485027, -0.83333758, -0.11644975],
       [ 1.04844655, -1.26806736, -1.16397627,  1.19877458,  0.02754332],
       [-0.22975709,  1.3323199 ,  5.85246505, -0.48766182, -0.17004027],
       [-0.01562696,  0.91386678,  0.95677187,  0.37599648, -0.15328046],
       [ 0.84447674, -0.10237652, -0.39475576,  1.30963292, -0.11327947],
       [ 0.50695631,  1.00353531, -0.05687385,  0.85431235, -0.13255084],
       [ 0.81381612,  0.43563464,  0.08690568,  0.3964251 , -0.14065717],
       [ 1.14610834, -0.31160308, -0.5385353 ,  1.09433673, -0.10710397],
       [ 1.5545122 , -1.47729392, -1.27181093,  0.5649958 ,  0.08505796],
       [ 1.00891017, -0.16215553, -0.5

In [166]:
# Huấn luyện mô hình
# model = RandomForestRegressor(random_state=42)
# model.fit(X_train, y_train)
models = {
    'LinearRegression': LinearRegression(),
    # 'RandomForest': RandomForestRegressor(random_state=42),
    # 'XGBoost': XGBRegressor(random_state=42)
}

metrics = {}

for name, model in models.items():
    # Huấn luyện
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Đánh giá
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    metrics[name] = {
        'MSE': mse,
        'R2': r2
    }

# 7. In kết quả
print("Model Evaluation Results:")
for name, metrics in metrics.items():
    print(f"{name}: MSE={metrics['MSE']:.4f}, R2={metrics['R2']:.4f}")

Model Evaluation Results:
LinearRegression: MSE=3.4232, R2=0.2181


In [167]:
# # Mức độ quan trọng của các đặc trưng
# importances = model.coef_
# features = X.columns

# # Vẽ biểu đồ mức độ quan trọng
# plt.figure(figsize=(10, 6))
# plt.barh(features, importances, color='skyblue')
# plt.xlabel('Importance')
# plt.title('Feature Importance')
# plt.show()

In [168]:
results = {}

results = {  
    'y_val': y_test,  
    'y_pred': y_pred  
} 

results_df = pd.DataFrame(results)
results_df.to_csv("Predict/train_predicted_TH.csv", index=False)

In [169]:
# Nhóm theo 'username' và tính toán các đặc trưng tổng hợp
# Mã hóa dữ liệu
# label_encoder = LabelEncoder()

# train_test_data['status_encoded'] = label_encoder.fit_transform(train_test_data['status'])

status_mapping = {
    "Compilation Error": 0,
    "Syntax Error": 0,
    "pending": 1,
    "SCORE": 2
}

# Áp dụng ánh xạ
predict_data['status_encoded'] = predict_data['status'].map(status_mapping)

predict_data['submission_count'] = predict_data.groupby('username')['is_final'].transform('count')
predict_data['avg_pre_score'] = predict_data.groupby('username')['pre_score'].transform('mean')

predict_data['avg_score_per_submission'] = predict_data['avg_pre_score'] / (predict_data['submission_count'] + 1e-5)  # Tránh chia cho 0


# Nhóm theo 'username' và tính toán các đặc trưng tổng hợp
agg_data = predict_data.groupby('username').agg({
    'avg_pre_score': 'mean',  # Trung bình điểm trước-
    'submission_count': 'mean',  # Trung bình thời gian nộp
    'avg_score_per_submission': 'mean', # Trung bình điểm trên tổng số lần nộp
    'status_encoded': 'mean',  # Trung bình trạng thái (hoặc có thể tính số lần xuất hiện nếu cần)
    'is_final': lambda x: (x == 1).sum()  # Đếm số lần giá trị bằng 1 trong cột is_final
}).reset_index()

# Cập nhật bộ dữ liệu với các đặc trưng tổng hợp
agg_data['TH'] = predict_data.groupby('username')['TH'].mean().values

# Tiếp theo, bạn có thể thực hiện các bước xử lý dữ liệu, chuẩn hóa và chia tách bộ dữ liệu như sau
features = ['status_encoded', 'is_final', 'submission_count', 'avg_pre_score', 'avg_score_per_submission']
# features = ['username','status_encoded', 'is_final', 'submission_count', 'avg_pre_score', 'avg_score_per_submission']

# Xử lý chuẩn hóa dữ liệu
X = agg_data[features]

scaler = StandardScaler()
X = scaler.fit_transform(X)



C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1891784436.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['status_encoded'] = predict_data['status'].map(status_mapping)
C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1891784436.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['submission_count'] = predict_data.groupby('username')['is_final'].transform('count')
C:\Users\Admin\AppData\Local\Temp\ipykernel_189920\1891784436.py:18: SettingWithCopyWarning: 
A value is tr

In [170]:
X

array([[-0.3295028 , -0.43430465, -0.69645451,  0.46726545, -0.09872688],
       [-0.66709268, -0.34842652, -0.34947888, -0.13019127, -0.13981776],
       [ 1.48215965, -1.63659846, -1.42194899,  1.00542834,  1.21951225],
       ...,
       [ 0.1520871 ,  1.16875376,  1.96106333, -0.15449169, -0.17919424],
       [-0.12392408, -0.80644321, -0.74376937, -0.58133758, -0.12101832],
       [ 0.57405817,  0.96837146,  1.66140257,  0.72853785, -0.17168772]])

In [171]:
agg_data['TH'] = model.predict(X)


In [172]:
agg_data

,username,avg_pre_score,submission_count,avg_score_per_submission,status_encoded,is_final,TH
0,0134f9f410c65ad0e8c2254a7e9288670e02a183,5952.760000,100.0,59.527594,1.520000,47,6.332547
1,035f97702f2c01d26ab1fae8f39ea2f98a0caa3c,5035.763889,144.0,34.970580,1.430556,50,6.421683
2,06024da8147b5cf6105beb4c403f67cf1ff0aea1,6778.750000,8.0,847.342691,2.000000,5,6.762381
3,0aaebc88f6106684d6993c156104c1ef36cf94e0,4065.369792,192.0,21.173800,1.552083,50,7.101332
4,0bf111a9caedf02804f6991792490e63bc21058a,6355.753769,199.0,31.938460,1.768844,82,7.820801
...,...,...,...,...,...,...,...
729,feb8a2859a011c59efd22ed419cb69288fe03627,5581.081081,74.0,75.420004,1.918919,34,7.285267
730,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,6204.557692,260.0,23.863683,1.661538,78,7.653592
731,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,4998.466819,437.0,11.438139,1.647597,103,8.760921
732,ff3fa2ec64294f37ae968159f810ebeda7966c51,4343.329787,94.0,46.205631,1.574468,34,6.574499


In [173]:
# Kiểm tra kết quả và chọn hai cột cần xuất
output = agg_data[['username', 'TH']]

# Xuất file kết quả
output = agg_data[['username', 'TH']].to_csv("Predict/predicted_TH.csv", index=False)